In [1]:
import warnings
warnings.filterwarnings('ignore')
import pyterrier as pt
if not pt.started():
    pt.init()

import os
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 100)
import glob
import ir_measures
from ir_measures import * # imports all supported measures, e.g., AP, nDCG, RR, P

Java started and loaded: pyterrier.java, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]
/tmp/ipykernel_38877/4087205142.py:5: DeprecationWarning: Call to deprecated method pt.init(). Deprecated since version 0.11.0.
java is now started automatically with default settings. To force initialisation early, run:
pt.java.init() # optional, forces java initialisation
  pt.init()


In [9]:
dataset_name = 'msmarco-passage'
eval_ds_name = 'dev'
dataset = pt.get_dataset(f'irds:{dataset_name}')
eval_dataset = pt.get_dataset(f'irds:{dataset_name}/{eval_ds_name}')
topics = eval_dataset.get_topics()
qrels = eval_dataset.get_qrels()

In [30]:
import pandas as pd
root_dir = f'/root/retrievability-bias'
nfs_dir = f'/nfs/resources/cxj/retrievability-bias'
modelname = 't5-base-msmarco-epoch-5'
ranker='tctcolbert'
inx = 3

import pyterrier_dr
import shutil
import os
import tqdm
model = pyterrier_dr.TctColBert('castorini/tct_colbert-v2-hnp-msmarco')

In [3]:
# csv = f'{nfs_dir}/bm25/{modelname}_unprunned_df.csv'

csv = f'{nfs_dir}/{modelname}_prunned_{30 * threshold}_df.csv'
df = pd.read_csv(csv, index_col=0).reset_index()

In [4]:
del df['prob']

In [5]:
df[['docno','text']] = df[['docno','text']].astype(str)

In [6]:
df.shape

(884183, 2)

In [ ]:
# df3 = pd.DataFrame([
#    {'docno': '0', 'text': 'The Five Find-Outers and Dog, also known as The Five Find-Outers, is a series of children\'s mystery books written by Enid Blyton.'},
#    {'docno': '1', 'text': 'City is a 1952 science fiction fix-up novel by American writer Clifford D. Simak.'},
#  ])

In [ ]:
df.head()

In [7]:
rng = [str(docno) for docno in range(8840800, 8840985)]
iterable_dict = df[df['docno'].isin(rng)].to_dict(orient="records")

In [ ]:
# for record in iterable_dict:
#     print(record)

In [8]:
# index_path = f"{root_dir}/{modelname}-{ranker}-index-inx-{inx}_test.flex"

# if os.path.exists(index_path):
#     shutil.rmtree(index_path)
#     print(f'existing index file at {index_path} removed')

# print(f'indexing into {index_path}')
# index = pyterrier_dr.FlexIndex(f'{index_path}')

# # idx_pipeline = DocumentFilter(qual_signal=qual_signal, threshold=threshold) >> model >> index
# idx_pipeline =  model >> index

# # idx_pipeline.index(df3)
# idx_pipeline.index(iterable_dict)

# print(f'indexing {index_path} done')

existing index file at /root/retrievability-bias/t5-base-msmarco-epoch-5-tctcolbert-index-threshold-3_test.flex removed
indexing into /root/retrievability-bias/t5-base-msmarco-epoch-5-tctcolbert-index-threshold-3_test.flex


indexing: 0dvec [00:00, ?dvec/s]

8840801 -----> 8840981


indexing: 25dvec [00:01, 13.65dvec/s]

indexing /root/retrievability-bias/t5-base-msmarco-epoch-5-tctcolbert-index-threshold-3_test.flex done


In [12]:
topics.shape

(101093, 2)

In [28]:
threshold = -0.15585532486438755
index_path = f"{nfs_dir}/{ranker}/{modelname}-{ranker}-index-threshold-{threshold}.flex"
index = pyterrier_dr.FlexIndex(index_path)
retr_pipeline = model >> index.torch_retriever() % 100

In [ ]:
retr_pipeline.transform(topics[:2])

In [31]:
bm25_csv = f'{nfs_dir}/bm25/df_bm25_{inx * 30}.csv'
bm25_df = pd.read_csv(bm25_csv, index_col=0).reset_index()
bm25_df[['qid', 'docno']] = bm25_df[['qid', 'docno']].astype(str)

In [41]:
retr_pipeline = pt.text.get_text(dataset, 'text', verbose=True) >> model
df5 = retr_pipeline.transform(bm25_df[:101])




IRDSTextLoader: 100%|██████████| 101/101 [00:00<00:00, 5625.38d/s]


In [42]:
df5

,qid,docid,docno,score,query,text,rank
0,1048578,730881,7187239,80.703613,cost of endless pools swim spa,"Endless Pools offers a family of swimming machines and therapy pools starting as low as $7,400. ...",1
1,1048578,730880,7187234,81.498634,cost of endless pools swim spa,Endless pools and swim spas are available in a number of different price brackets. For the brand...,0
2,1048578,211016,2078215,79.260941,cost of endless pools swim spa,"How much does a swim spa cost. 1 A top of the line, branded swim spa 9 feet by 16 feet with the...",3
3,1048578,693636,6802211,77.482460,cost of endless pools swim spa,"What Is A Swim Spa. A swim spa is the combination of a pool and a spa, in which you swim against...",59
4,1048578,674218,6599701,76.676346,cost of endless pools swim spa,"Also known as: a spool pool. When discussing pools, a spool is a portmanteau word or a blend of ...",77
5,1048578,442407,4332303,79.635735,cost of endless pools swim spa,"As mentioned in the introduction, a swim spa can cost less than half that of a standard swimming...",2
6,1048578,679283,6656280,76.913872,cost of endless pools swim spa,That being said you should never add Epsom salts to your spa or swim spa. Salt levels above 1500...,71
7,1048578,679282,6656275,76.737953,cost of endless pools swim spa,That being said you should never add Epsom salts to your spa or swim spa. Here is why: Salt leve...,75
8,1048578,767320,7583795,75.883705,cost of endless pools swim spa,Confidence votes 82. Cyanuric acid is generally called chlorine stabilizer and it is used to pre...,90
9,1048578,29701,290325,77.006454,cost of endless pools swim spa,"Traditional lap pools are about 10 feet wide, though widths between 8 and 10 feet are common. (I...",66
